In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show#, output_file #Descomentar para guardar el archivo .html
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

# Círculo

In [3]:
x1 = np.array([np.random.uniform(-6,6) for i in range(10000)])
x2 = np.array([np.random.uniform(-6,6) for i in range(10000)])

In [4]:
x1cuad = np.array([i**2 for i in x1])
x2cuad = np.array([i**2 for i in x2])

In [5]:
X = np.array([[x1cuad[i],x2cuad[i]] for i in range(len(x1))])

In [5]:
X = np.array([[x1[i],x2[i],x1cuad[i],x2cuad[i]] for i in range(len(x1))])

In [6]:
circulo = np.array([1 if np.sqrt(x1[i]**2+x2[i]**2)<=3 else 0 for i in range(len(x1))])

In [7]:
circulo = circulo.reshape(-1,1)

In [8]:
p = figure(width=800, plot_height=800, title=None)
p.circle(x1,x2,color=["red" if np.sqrt(x1[i]**2+x2[i]**2)<=3 else "blue" for i in range(len(x1))])
show(p)

In [9]:
input_size=4
hidden_layer_size=3
output_layer_size=1
#Los placeholders son para los datos de entrada
x = tf.placeholder(tf.float32, shape=[len(x1),input_size ])
y_ = tf.placeholder(tf.float32, shape=[len(circulo),output_layer_size ]) #la y real
#La variables para lo que se va a ir calculando y modificando en el camino
W_layer1=tf.Variable(tf.random_uniform([input_size,hidden_layer_size], -1, 1), name="W_layer1")
W_layer2=tf.Variable(tf.random_uniform([hidden_layer_size,output_layer_size],-1, 1), name="W_layer2")
b_layer1 = tf.Variable(tf.zeros([hidden_layer_size]), name="b_layer1")
b_layer2 = tf.Variable(tf.zeros([output_layer_size]), name="b_layer2")

In [10]:
y = tf.sigmoid(tf.matmul(x,W_layer1)+b_layer1) #la de nuestra
hip = tf.sigmoid(tf.matmul(y, W_layer2)+b_layer2)
#lossfn = tf.reduce_mean(tf.reduce_sum((y_-y)**2)) #cuadratico
lossfn= tf.reduce_mean(( (y_ * tf.log(hip)) +  
  ((1 - y_) * tf.log(1.0 - hip)) ) * -1) # cross entropy
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(lossfn)

In [11]:
init = tf.initialize_all_variables()
sess = tf.Session()

In [12]:
sess.run(init)
for i in range(50000):
    sess.run(train_step, feed_dict={x: X, y_: circulo})

In [13]:
correct_prediction = tf.equal(tf.round(hip),y_) # aca estams definiendo otras operacioes
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: X, y_: circulo})) # las corremos

0.9984


In [14]:
print(sess.run(hip,feed_dict={x: X, y_: circulo}))

[[ 0.33408582]
 [ 0.00421073]
 [ 0.00155746]
 ..., 
 [ 0.00159921]
 [ 0.00156458]
 [ 0.73324448]]


In [15]:
circulo[22:32]

array([[1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0]])

In [16]:
print(sess.run(hip,feed_dict={x: X, y_: circulo}))[22:32]

[[ 0.65173048]
 [ 0.00830473]
 [ 0.00155757]
 [ 0.98568034]
 [ 0.99793589]
 [ 0.00156881]
 [ 0.00167561]
 [ 0.05233412]
 [ 0.99850887]
 [ 0.16225867]]


In [17]:
Y = sess.run(hip,feed_dict={x: X, y_: circulo})

In [18]:
Y = np.round(Y)

In [19]:
p = figure(width=800, plot_height=800, title=None)
p.circle(X[:,0],X[:,1],color=["red" if i==1 else "blue" for i in Y])
p.title.text = "Cículo predicho"
p.xaxis.axis_label = "X1"
p.yaxis.axis_label = "X2"
show(p)  